In [1]:
import weave
import json
import openai
import new_api

2024-06-22 14:01:25.048 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-06-22 14:01:25.050 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-06-22 14:01:25.050 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
def load_jsonl_data(fname):
    result = []
    for l in open(fname):
        result.append(json.loads(l))
    return result

In [3]:
problems = load_jsonl_data('connections_prompts.jsonl')

In [4]:
@weave.op()
def solve_connections(prompt, words):
    response = openai.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'user', 'content': prompt.format(words=words)}],
        response_format={ "type": "json_object"}
    )
    return json.loads(response.choices[0].message.content)

In [5]:
prompt_a = '''Solve this NYTimes Connections problem: {words}.
Please output your solution as a json object with two fields:
- step_by_step_reasoning: str
- groups: list of objects with keys words (list) reason (str)
'''

prompt_b = '''Solve this NYTimes Connections problem: {words}.

You should propose at least 10 potential groupings of words

Please output your solution as a json object with two fields:
- step_by_step_reasoning: str.
    First, propose at least 10 potential groupings of words.
    Next, score each grouping for how likely it is to be a Connections group.
    Then, propose at least two potential solutions, scoring them.
- groups: Final list of objects with keys words (list) reason (str)
'''

prompt_c = '''Solve this NYTimes Connections problem: {words}.

You must follow these steps:
1. Generate a preliminary set of 10 word groups.
2. Check each group to remove words not present in the original list or clearly irrelevant.
3. Ensure uniqueness by comparing every word across all groups to eliminate overlaps.
4. Validate each group by comparing to traditional Connections categories and scoring each group's probability.
5. Propose multiple distinct final solutions, each containing unique sets of words, avoiding conflicts.

Output should be structured as json:
- step_by_step_reasoning: Extensive reasoning for group formulation, validation, and solution selection.
- groups: List of finalized groups, each with a list of words and a reason.
'''

# result = solve_connections(prompt_a, problems[4]['words'])
# from rich import print
# print(result['step_by_step_reasoning'])
# print(list(result['groups']))

In [6]:
print(problems[4])

{'words': ['glum', 'prime', 'low', 'hulu', 'green', 'plum', 'blue', 'down', 'peacock', 'mayo', 'scarlet', 'ketchup', 'relish', 'netflix', 'mustard', 'tartar'], 'solution': {'groups': [{'words': ['ketchup', 'mayo', 'relish', 'tartar'], 'reason': 'condiments'}, {'words': ['hulu', 'netflix', 'peacock', 'prime'], 'reason': 'streaming services'}, {'words': ['blue', 'down', 'glum', 'low'], 'reason': 'synonyms for sad'}, {'words': ['green', 'mustard', 'plum', 'scarlet'], 'reason': 'clue characters'}]}}


In [7]:
@weave.op()
def score(gen_solution, solution):
    solution_set = set(','.join(sorted(g['words'])) for g in solution['groups'])
    gen_solution_set = set(','.join(sorted(g['words'])) for g in gen_solution['groups'])
    return len(solution_set.intersection(gen_solution_set))

In [8]:
#score(result, problems[1]['solution'])

In [9]:
prompts = [prompt_a, prompt_b]

data = []
for p in prompts:
    for prob in problems[:10]:
        data.append({**prob, 'prompt': p})

ct = new_api.ComputeTable(data, trials=1)
ct.add_op(solve_connections)
ct.add_op(score, {'gen_solution': 'solve_connections'})

In [10]:
weave.init_local_client()

In [11]:
ct.fill_from_cache()

/Users/shawn/code2/weave-analysis/query.py:190: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_summed = df_renamed.groupby(axis=1, level=0).sum()
/Users/shawn/code2/weave-analysis/query.py:190: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_summed = df_renamed.groupby(axis=1, level=0).sum()
/Users/shawn/code2/weave-analysis/query.py:190: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_summed = df_renamed.groupby(axis=1, level=0).sum()
/Users/shawn/code2/weave-analysis/query.py:190: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_summed = df_renamed.groupby(axis=1, level=0).sum()
/Users/shawn/code2/weave-analysis/query.py:190: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis inste

rows                                      id                              trace_id  \
1  60ebf949-7db1-4ba4-ac7a-78295032bc7d  255e89a1-620c-4009-b2c6-00d710ad59e1   

  parent_id                                            op_name op_name.entity  \
1      None  weave:///none/none/op/solve_connections:xEBAYY...           none   

  op_name.project op_name.kind       op_name.name  \
1            none           op  solve_connections   

                               op_name.version input_refs exception  \
1  xEBAYYRXSM8KWV8ggPilnGX8yhg8SZMXaePhRMEG13w         []      None   

                                       inputs.prompt  \
1  Solve this NYTimes Connections problem: {words...   

                                        inputs.words  \
1  [nets, return, heat, jazz, mom, shift, kayak, ...   

                       output.step_by_step_reasoning  \
1  First, identify groups of words that share a c...   

                                       output.groups  
1  [{'words': ['rain', 's

/Users/shawn/code2/weave-analysis/query.py:190: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_summed = df_renamed.groupby(axis=1, level=0).sum()
/Users/shawn/code2/weave-analysis/query.py:190: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_summed = df_renamed.groupby(axis=1, level=0).sum()
/Users/shawn/code2/weave-analysis/query.py:190: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_summed = df_renamed.groupby(axis=1, level=0).sum()
/Users/shawn/code2/weave-analysis/query.py:190: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_summed = df_renamed.groupby(axis=1, level=0).sum()
/Users/shawn/code2/weave-analysis/query.py:190: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis inste

rows                                       id  \
2   d80b33e9-fbf2-499a-9e9a-e37cf095bedc   
12  f283e0af-4042-4368-849f-04a9ae2c6af2   

                                trace_id parent_id  \
2   dae3491d-4be8-437e-b6c3-929fbe967357      None   
12  c4a20dae-4602-464a-9f18-216ebc3689f3      None   

                                              op_name op_name.entity  \
2   weave:///none/none/op/score:YdIu5IJhP5Ho1UQiSp...           none   
12  weave:///none/none/op/score:YdIu5IJhP5Ho1UQiSp...           none   

   op_name.project op_name.kind op_name.name  \
2             none           op        score   
12            none           op        score   

                                op_name.version  \
2   YdIu5IJhP5Ho1UQiSp8RTAlKEVtzObNyBmJQ0D19Ed8   
12  YdIu5IJhP5Ho1UQiSp8RTAlKEVtzObNyBmJQ0D19Ed8   

                                           input_refs  output exception  \
2   [weave:///none/none/call/f47ae214-eece-4a8a-a1...       2      None   
12  [weave:///none/none/call/1635

/Users/shawn/code2/weave-analysis/query.py:190: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_summed = df_renamed.groupby(axis=1, level=0).sum()
/Users/shawn/code2/weave-analysis/query.py:190: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_summed = df_renamed.groupby(axis=1, level=0).sum()
/Users/shawn/code2/weave-analysis/query.py:190: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_summed = df_renamed.groupby(axis=1, level=0).sum()
/Users/shawn/code2/weave-analysis/query.py:190: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_summed = df_renamed.groupby(axis=1, level=0).sum()
/Users/shawn/code2/weave-analysis/query.py:190: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis inste

In [14]:
ct.dataframe()

,words,solution,prompt,trial,solve_connections,score
0,"[nets, return, heat, jazz, mom, shift, kayak, ...","{'groups': [{'words': ['bucks', 'heat', 'jazz'...",Solve this NYTimes Connections problem: {words...,0,"{'step_by_step_reasoning': 'First, identify gr...",4
1,"[league, loafer, queue, are, pump, foot, why, ...","{'groups': [{'words': ['foot', 'league', 'mile...",Solve this NYTimes Connections problem: {words...,0,"{'step_by_step_reasoning': 'First, identify if...",2
2,"[amigo, wolf, cheek, pom, stooge, lab, king, p...","{'groups': [{'words': ['chow', 'gobble', 'scar...",Solve this NYTimes Connections problem: {words...,0,"{'step_by_step_reasoning': 'First, identify po...",2
3,"[puma, sweep, chicago, bat, iron, adidas, vacu...","{'groups': [{'words': ['cabaret', 'carousel', ...",Solve this NYTimes Connections problem: {words...,0,"{'step_by_step_reasoning': 'First, I noticed t...",2
4,"[glum, prime, low, hulu, green, plum, blue, do...","{'groups': [{'words': ['ketchup', 'mayo', 'rel...",Solve this NYTimes Connections problem: {words...,0,"{'step_by_step_reasoning': 'First, observe if ...",1
5,"[future, sister, q-tip, sin, chance, boardwalk...","{'groups': [{'words': ['baby', 'midnight', 'po...",Solve this NYTimes Connections problem: {words...,0,{'step_by_step_reasoning': 'Let's start by ide...,1
6,"[calf, star, ankle, shin, silver, cray, john, ...","{'groups': [{'words': ['calf', 'cub', 'joey', ...",Solve this NYTimes Connections problem: {words...,0,"{'step_by_step_reasoning': 'First, I scanned t...",1
7,"[checkers, twin, locksmith, ruby, backgammon, ...","{'groups': [{'words': ['full', 'king', 'queen'...",Solve this NYTimes Connections problem: {words...,0,"{'step_by_step_reasoning': 'First, identify th...",4
8,"[beetle, squash, termite, cricket, beet, polo,...","{'groups': [{'words': ['cami', 'halter', 'tank...",Solve this NYTimes Connections problem: {words...,0,{'step_by_step_reasoning': 'To solve the puzzl...,4
9,"[orange, jay, goat, turkey, twins, scales, dat...","{'groups': [{'words': ['chad', 'georgia', 'jor...",Solve this NYTimes Connections problem: {words...,0,{'step_by_step_reasoning': 'To solve this puzz...,2


In [12]:
raise Exception

Exception: 

In [ ]:
#from rich.live import Live
import tqdm
for delta in tqdm.tqdm(ct.execute()):
    pass
    #live.update(ct.rich_table())
    #print(ct.rich_table())

In [ ]:
res = ct.dataframe()
res

In [ ]:
res['words_s'] = res['words'].apply(lambda x: json.dumps(x))

In [ ]:
res.pivot(index=('words_s', 'trial'), columns='prompt')

In [ ]:
res.pivot(index=('words_s', 'trial'), columns='prompt')['score'].groupby(level='words_s').mean()

In [ ]:
res.pivot(index=('words_s', 'trial'), columns='prompt')['score'].mean()

In [ ]:
@weave.op()
def explain(rationale, problem, prompt):
    prompt = f'''Please explain how to improve the following LLM prompt, given a failing example.
Generate 5 new prompt variants, ranging from simple improvements to extreme changes.
You can ask LLMs to perform looping and other complex control flow operations.
Don't limit your changes to just wording.
Pay careful attention to mistakes made in the provided response.
We need to update the prompt to prevent similar mistakes.
For example, we might notice that the LLM chose a correct grouping category, but included the wrong words in that group, because there are 5 potential words for the group. Noticing this, we might ask the LLM to figure out potential categories first, and then for each word output the categories it might match, before proceeding. This is just one example of the kind of clever change I want you to suggest.

Prompt used to generate solution with LLM:
{prompt}

Problem:
{problem}

LLM generated rationale:
{rationale}
'''
    response = openai.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'user', 'content': prompt}]
    )
    return response.choices[0].message.content

In [ ]:
result = explain("""First, I will identify potential groupings of words based on common themes or associations. Here are at least 10 
potential groupings:

1. Streaming Services: 'netflix', 'hulu', 'prime', 'peacock'
2. Colors: 'green', 'blue', 'scarlet', 'plum'
3. Condiments: 'ketchup', 'mustard', 'relish', 'tartar'
4. Emotions: 'glum', 'low', 'down', 'blue'
5. Fruit: 'plum', 'mayo' (plays on 'may', close to the month 'April'), 'scarlet' (thinking of 'Red Delicious 
Apples'), 'mango' (not in the list)
6. Status/State: 'prime', 'down', 'low', 'high' (not present)
7. Words related to 'Level': 'prime', 'low', 'down', 'high (not present)
8. Food items: 'relish', 'ketchup', 'mustard', 'tartar'
9. Bird-related: 'peacock' (actual bird), 'plover' (not in the list but related to birds), 'jay' (not in the list),
'crane' (not present)
10. Online Platforms: 'netflix', 'hulu', 'prime', 'peacock' (already seen in Streaming Services)

Now I will score each grouping for how likely it is to be a Connections group on a scale of 1 to 10, with 10 being 
highly likely.

1. Streaming Services: 10/10
2. Colors: 8/10
3. Condiments: 10/10
4. Emotions: 8/10
5. Fruit: 4/10
6. Status/State: 6/10
7. Words related to 'Level': 5/10
8. Food items: 10/10
9. Bird-related: 2/10
10. Online Platforms: 10/10

Based on these groupings and scores, let me propose at least two potential solutions and score them.

Potential Solution 1:
Group 1: Streaming Services - 'netflix', 'hulu', 'prime', 'peacock' -> Score: 10/10
Group 2: Colors - 'green', 'blue', 'scarlet', 'plum' -> Score: 8/10
Group 3: Condiments - 'ketchup', 'mustard', 'relish', 'tartar' -> Score: 10/10
Group 4: Emotions - 'glum', 'low', 'down', 'blue' -> Score: 8/10
Total Score: 36/40

Potential Solution 2:
Group 1: Streaming Services - 'netflix', 'hulu', 'prime', 'peacock' -> Score: 10/10
Group 2: Colors - 'green', 'blue', 'scarlet', 'plum' -> Score: 8/10
Group 3: Condiments - 'ketchup', 'mustard', 'relish', 'tartar' -> Score: 10/10
Group 4: Status/State - 'prime', 'low', 'down', 'glum' -> Score: 6/10
Total Score: 34/40

Therefore, Potential Solution 1 appears to be the most promising.""", problems[4], prompt_b)

In [ ]:
print(result)